In [ ]:
write_images = False


wirte_output_txt = False 
# Specify everytime Simulation is called 
# WARNING --> Set to False when running more then 10 simulations 
#            (otherwise it will be super slow and might crash)

In [ ]:
import numpy as np

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.core.initial_designs import RandomDesign

import GPy
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

import matplotlib.pyplot as plt
import mlai.plot as plot

In [ ]:
%run Missile_utils.ipynb

In [ ]:
simulation_output = 'range' 
# We divide by 1000 to avoid dealing with too large numbers

In [ ]:
run_grid_simulation = True # If true takes much longer and does 3D plots and so on for MODEL with 2 FEATS


We consider missiles with only 1 stage

In [ ]:

basic_param_spaces = {
    'payload':  [10, 2410],
    'missilediam':  [0.1, 9.9],
    'rvdiam':  [0.1, 9.9],
    'estrange': [100, 4900], 
    'fuelmass': [500, 6000], # [500, 6000], 
    'drymass':  [1000, 3000],
    'Isp0':  [100, 800],# [100, 800],
    'thrust0':  [10000, 69000],
}

In [ ]:
from sklearn.metrics import mean_squared_error
import math

def compute_rmse(y_actual, y_predicted):
    MSE = mean_squared_error(y_actual, y_predicted)
    RMSE = math.sqrt(MSE)
 
    return RMSE

def evaluate_prediction(y_actual, y_predicted):
    return compute_rmse(y_actual, y_predicted)
    

# 1. Two params

In [ ]:
m2_param_1 = 'fuelmass'
m2_domain_param_1 = basic_param_spaces[m2_param_1] 
m2_param_2 = 'Isp0'
m2_domain_param_2 = basic_param_spaces[m2_param_2] 

m2_space = ParameterSpace(
          [ContinuousParameter(m2_param_1, *m2_domain_param_1), 
           ContinuousParameter(m2_param_2, *m2_domain_param_2),
          ])

custom_param_names = [m2_param_1, m2_param_2]

In [ ]:
def run_missile_sim(custom_params):
    """
    Recives in input an array of custom parameters.
    Each row represents a set of different parameters
    Each column is a different parameter (#cols = len(custom_param_names))
    """
    default_params_IRAQ = {
        'payload':500,
        'missilediam':0.88,
        'rvdiam':0,
        'estrange':600,
        'numstages':1,
        'fuelmass':[0,5600],
        'drymass':[0,1200],
        'Isp0':[0,226],
        'thrust0':[0,9177.4]
    }
    
    
    y = np.zeros((custom_params.shape[0], 1))
    for i in range(custom_params.shape[0]):
        params_to_use = default_params_IRAQ
        # Overwrite default param variables
        for j in range(custom_params.shape[1]):
            param_name = custom_param_names[j]
            if param_name in ['fuelmass', 'drymass', 'Isp0', 'thrust0']:
                params_to_use[param_name][1] = custom_params[i,j]
            else:
                params_to_use[param_name] = custom_params[i, j]
                
            if j==0:
                print('\nNew simulation \n')
            str_to_print = param_name + ': ' + str(custom_params[i,j])
            print(str_to_print)
                
        # Run simulation
        output_path = 'results/results_' + str(i) + '.txt'
        sim_output = run_one_sim(
            numstages=params_to_use["numstages"], 
            fuelmass=params_to_use["fuelmass"], 
            drymass=params_to_use["drymass"], 
            thrust0=params_to_use["thrust0"], 
            Isp0=params_to_use["Isp0"], 
            payload=params_to_use["payload"],  
            missilediam=params_to_use["missilediam"],  
            rvdiam=params_to_use["rvdiam"], 
            est_range=params_to_use["estrange"], 
            output_path=output_path, 
            simulation_output=simulation_output,
        )
        
        y[i, 0] = sim_output
    return y


def neg_run_missile_sim(custom_params):
    return -run_missile_sim(custom_params)
    
    

## 1. Experimental design

### Use model-free experimental design to start
 (RandomDesign or Latin Design)

In [ ]:
wirte_output_txt = True


# from emukit.core.initial_designs.latin_design import LatinDesign
# design = LatinDesign(parameter_space) 

m2_design = RandomDesign(m2_space)
m2_x = m2_design.get_samples(3*2)
m2_y = neg_run_missile_sim(m2_x)

In [ ]:
# Build model
m2_var_kernel = (100)**2 
m2_lengthscale = 100 # 100 # 1
m2_var_noise = 1e-5 # small value

#kern = GPy.kern.RBF(input_dim=2, lengthscale=lengthscale, variance =var_kernel)  # , lengthscale=0.08, variance=20
# kern = GPy.kern.Matern32(input_dim=1)
# kern = GPy.kern.Linear(input_dim=1)

constrain_lengthscale = True

m2_rbf_kern = GPy.kern.RBF(input_dim=2, lengthscale=m2_lengthscale)
if constrain_lengthscale:
    m2_rbf_kern.lengthscale.constrain_bounded(m2_lengthscale, m2_lengthscale*1e12)

# m2_kern = m2_rbf_kern + \
#     GPy.kern.Linear(input_dim=2)
m2_kern = (GPy.kern.RBF(input_dim=2, lengthscale=500) * \
           GPy.kern.RBF(input_dim=2, lengthscale=100)) * \
    GPy.kern.Linear(input_dim=2)
# m2_kern = m2_rbf_kern

m2_model_gpy = GPRegression(m2_x,m2_y, kernel=m2_kern)
m2_model_gpy.kern.variance =  m2_var_kernel 
m2_model_gpy.likelihood.variance.fix(m2_var_noise)  

display(m2_model_gpy)




In [ ]:
# m2_model_gpy_opt = m2_model_gpy
# m2_model_gpy_opt.optimize()
# m2_model_gpy_opt.plot()

In [ ]:
m2_model_emukit = GPyModelWrapper(m2_model_gpy)
m2_model_emukit.optimize()

In [ ]:
display(m2_model_gpy)

In [ ]:
# Create data for plot
wirte_output_txt = False
nr_points_plot = 101
m2_param_1_x_plot = np.linspace(m2_space.parameters[0].min, m2_space.parameters[0].max, nr_points_plot)[:, None]
m2_param_2_x_plot = np.linspace(m2_space.parameters[1].min, m2_space.parameters[1].max, nr_points_plot)[:, None]
m2_x_plot_mesh, m2_y_plot_mesh = np.meshgrid(m2_param_1_x_plot, m2_param_2_x_plot)
m2_x_plot = np.array([m2_x_plot_mesh, m2_y_plot_mesh]).T.reshape(-1,2)


# TEMP read data from txt
# np.savetxt('test1.txt', a, fmt='%f')
# m2_y_plot = np.loadtxt('m2_y_plot_neg.txt', dtype=float)[:,None]

if run_grid_simulation:
    m2_y_plot = neg_run_missile_sim(m2_x_plot) # TAKES LONG TIME    
    m2_Z = m2_y_plot.reshape(m2_x_plot_mesh.shape)




In [ ]:
m2_x

In [ ]:
# Compute current prediction
m2_mu_plot_grid_pred1, var_plot_grid_pred1 = m2_model_emukit.predict(m2_x_plot)
m2_mu_plot_pred1 = m2_mu_plot_grid_pred1.reshape(m2_x_plot_mesh.shape)
m2_var_plot_pred1 = var_plot_grid_pred1.reshape(m2_x_plot_mesh.shape)

if run_grid_simulation:
    m2_rmse = evaluate_prediction(y_actual=m2_y_plot, y_predicted=m2_mu_plot_grid_pred1)
    print("RMSE m2 (before experiment design loop): ", m2_rmse)


In [ ]:
if run_grid_simulation:
    # 3D Plot
    add_bands = False
    
    # REVERSE
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # True surface
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_Z).transpose(), 
                           alpha = .5, 
                           label='target function',
                           color='black'
                          )
    # Mean predicted
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_mu_plot_pred1).transpose(), 
                           alpha = .5, 
                           label='model', # Mean
                           color='royalblue'
                          )
    # True points observed
    ax.scatter(m2_x[:,1], m2_x[:,0], m2_y, marker='o', color='red')

    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    ax.set_zlabel('Target function')
    plt.tight_layout()



In [ ]:
if run_grid_simulation:
    from matplotlib import colors
    divnorm=colors.TwoSlopeNorm(vcenter=0.) # vmin=-5., vcenter=0., vmax=10

    ## Heatmaps
    extents = [m2_space.parameters[1].min, m2_space.parameters[1].max, 
               m2_space.parameters[0].min, m2_space.parameters[0].max]

    # True values
    fig, ax = plt.subplots()
    im = ax.imshow(m2_Z, extent=extents, aspect='auto',  origin='lower')
    ax.set_title('Target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()

    # Model
    fig, ax = plt.subplots()
    im = ax.imshow(m2_mu_plot_pred1, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Model (mean)')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()


    # Difference
    fig, ax = plt.subplots()
    vmin = (m2_mu_plot_pred1-m2_Z).min()
    vmax = (m2_mu_plot_pred1-m2_Z).max()
    vmin_max = max(abs(vmin), abs(vmax))
    divnorm=colors.TwoSlopeNorm(vcenter=0., vmin=-vmin_max, vmax=vmin_max) # vmin=-5., vcenter=0., vmax=10
    im = ax.imshow(m2_mu_plot_pred1-m2_Z, extent=extents, aspect='auto', cmap="bwr", norm=divnorm, origin='lower')
    ax.set_title('Difference between model and target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    # Add points where simulation evaluated
    ax.plot(m2_x[:,1], m2_x[:,0], 'ro')
    fig.colorbar(im, ax=ax)
    fig.show()



In [ ]:
m2_x

### Use the model created for model-based experimental design
use the model to decide which are the best points to collect using some data collection criteria (acquisition function).

In [ ]:
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.experimental_design.acquisitions import IntegratedVarianceReduction, ModelVariance


In [ ]:
# help(ExperimentalDesignLoop)

In [ ]:
m2_2_model_emukit = m2_model_emukit

In [ ]:
wirte_output_txt = False

integrated_variance = IntegratedVarianceReduction(space=m2_space,
                                                  model=m2_2_model_emukit)
m2_ed = ExperimentalDesignLoop(space=m2_space, 
                            model=m2_2_model_emukit, 
                            acquisition = integrated_variance,
                            batch_size = 1) 

m2_ed.run_loop(user_function=neg_run_missile_sim, stopping_condition=10*2)



In [ ]:
m2_2_model_emukit.X.shape


In [ ]:
m2_2_model_emukit.__dict__

In [ ]:
m2_ed.__dict__

In [ ]:
if run_grid_simulation:
    # Compute new prediction
    m2_mu_plot_grid_pred2, var_plot_grid_pred2 = m2_2_model_emukit.predict(m2_x_plot)
    m2_mu_plot_pred2 = m2_mu_plot_grid_pred2.reshape(m2_x_plot_mesh.shape)
    m2_var_plot_pred2 = var_plot_grid_pred2.reshape(m2_x_plot_mesh.shape)
    
    m2_2_rmse = evaluate_prediction(y_actual=m2_y_plot, y_predicted=m2_mu_plot_grid_pred2)
    print("RMSE m2 (post experiment design loop): ", m2_2_rmse)




In [ ]:
if run_grid_simulation:
    # 3D Plot
    add_bands = False

    
    # REVERSE
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # True surface
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_Z).transpose(), 
                           alpha = .5, 
                           label='target function',
                           color='black'
                          )
    # Mean predicted
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_mu_plot_pred2).transpose(), 
                           alpha = .5, 
                           label='model', # Mean
                           color='royalblue'
                          )
    # True points observed
    ax.scatter(np.array(m2_2_model_emukit.X)[:,1], 
               np.array(m2_2_model_emukit.X)[:,0], m2_2_model_emukit.Y, marker='o', color='red')

    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    ax.set_zlabel('Target function')
    plt.tight_layout()




In [ ]:
if run_grid_simulation:

    # REVERSE
    ## Heatmaps
    extents = [m2_space.parameters[1].min, m2_space.parameters[1].max, 
               m2_space.parameters[0].min, m2_space.parameters[0].max]

    # True values
    fig, ax = plt.subplots()
    im = ax.imshow(m2_Z, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()

    # Model
    fig, ax = plt.subplots()
    im = ax.imshow(m2_mu_plot_pred2, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Model (mean)')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()


    # Difference
    fig, ax = plt.subplots()
    vmin = (m2_mu_plot_pred2-m2_Z).min()
    vmax = (m2_mu_plot_pred2-m2_Z).max()
    vmin_max = max(abs(vmin), abs(vmax))
    divnorm=colors.TwoSlopeNorm(vcenter=0., vmin=-vmin_max, vmax=vmin_max) # vmin=-5., vcenter=0., vmax=10
    im = ax.imshow(m2_mu_plot_pred2-m2_Z, extent=extents, aspect='auto', cmap="bwr", norm=divnorm, origin='lower')
    ax.set_title('Difference between model and target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    # Add points where simulation evaluated
    ax.plot(np.array(m2_2_model_emukit.X)[:,1], np.array(m2_2_model_emukit.X)[:,0], 'ro')
    fig.colorbar(im, ax=ax)
    fig.show()



## 2. Maximization


In [ ]:
m2_model_gpy = GPRegression(m2_x,m2_y, kernel=m2_kern)
m2_model_gpy.kern.variance =  m2_var_kernel 
m2_model_gpy.likelihood.variance.fix(m2_var_noise)  

m2_model_emukit = GPyModelWrapper(m2_model_gpy)
m2_model_emukit.optimize()


In [ ]:
# Compute current prediction
m2_mu_plot_grid_pred1, var_plot_grid_pred1 = m2_model_emukit.predict(m2_x_plot)
m2_mu_plot_pred1 = m2_mu_plot_grid_pred1.reshape(m2_x_plot_mesh.shape)
m2_var_plot_pred1 = var_plot_grid_pred1.reshape(m2_x_plot_mesh.shape)

if run_grid_simulation:
    m2_rmse = evaluate_prediction(y_actual=m2_y_plot, y_predicted=m2_mu_plot_grid_pred1)
    print("RMSE m2 (before experiment design loop): ", m2_rmse)



In [ ]:
if run_grid_simulation:
    # 3D Plot
    add_bands = False
    
    # REVERSE
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # True surface
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_Z).transpose(), 
                           alpha = .5, 
                           label='target function',
                           color='black'
                          )
    # Mean predicted
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_mu_plot_pred1).transpose(), 
                           alpha = .5, 
                           label='model', # Mean
                           color='royalblue'
                          )
    # True points observed
    ax.scatter(m2_x[:,1], m2_x[:,0], m2_y, marker='o', color='red')

    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    ax.set_zlabel('Target function')
    plt.tight_layout()


In [ ]:
if run_grid_simulation:
    from matplotlib import colors
#     divnorm=colors.TwoSlopeNorm(vcenter=0.) # vmin=-5., vcenter=0., vmax=10

    # REVERSE
    ## Heatmaps
    extents = [m2_space.parameters[1].min, m2_space.parameters[1].max, 
               m2_space.parameters[0].min, m2_space.parameters[0].max]

    # True values
    fig, ax = plt.subplots()
    im = ax.imshow(m2_Z, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()

    # Model
    fig, ax = plt.subplots()
    im = ax.imshow(m2_mu_plot_pred1, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Model (mean)')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()


    # Difference
    vmin = (m2_mu_plot_pred1-m2_Z).min()
    vmax = (m2_mu_plot_pred1-m2_Z).max()
    vmin_max = max(abs(vmin), abs(vmax))
    divnorm=colors.TwoSlopeNorm(vcenter=0., vmin=-vmin_max, vmax=vmin_max) # vmin=-5., vcenter=0., vmax=10
    fig, ax = plt.subplots()
    im = ax.imshow(m2_mu_plot_pred1-m2_Z, extent=extents, aspect='auto', cmap="bwr", norm=divnorm, origin='lower')
    ax.set_title('Difference between model and target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    # Add points where simulation evaluated
    ax.plot(m2_x[:,1], m2_x[:,0], 'ro')
    fig.colorbar(im, ax=ax)
    fig.show()



### Use the model created for model-based bayes optimization
use the model to decide which are the best points to collect using some data collection criteria (that we call acquisition).

In [ ]:
m2_max_model_emukit = m2_model_emukit

In [ ]:
# Bayesian optimization using emulator
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

maxim_aquisition = ExpectedImprovement(model=m2_max_model_emukit)


bayesopt_loop = BayesianOptimizationLoop(model = m2_max_model_emukit,
                                         space = m2_space,
                                         acquisition = maxim_aquisition,
                                         batch_size = 1)

max_iterations = 5*2

bayesopt_loop.run_loop(neg_run_missile_sim, max_iterations) 



In [ ]:
results = bayesopt_loop.get_results()
results



In [ ]:
m2_max_model_emukit.X.shape

In [ ]:
m2_max_model_emukit.__dict__

In [ ]:
if run_grid_simulation:
    # Compute new prediction
    m2_mu_plot_grid_pred2, var_plot_grid_pred2 = m2_max_model_emukit.predict(m2_x_plot)
    m2_mu_plot_pred2 = m2_mu_plot_grid_pred2.reshape(m2_x_plot_mesh.shape)
    m2_var_plot_pred2 = var_plot_grid_pred2.reshape(m2_x_plot_mesh.shape)
    
    m2_max_rmse = evaluate_prediction(y_actual=m2_y_plot, y_predicted=m2_mu_plot_grid_pred2)
    print("RMSE m2 (post bayes opt loop): ", m2_max_rmse)




In [ ]:
if run_grid_simulation:
    # 3D Plot
    add_bands = False

    
    # REVERSE
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    # True surface
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_Z).transpose(), 
                           alpha = .5, 
                           label='target function',
                           color='black'
                          )
    # Mean predicted
    surf = ax.plot_surface(m2_y_plot_mesh, m2_x_plot_mesh, (m2_mu_plot_pred2).transpose(), 
                           alpha = .5, 
                           label='model', # Mean
                           color='royalblue'
                          )
    # True points observed
    ax.scatter(np.array(m2_max_model_emukit.X)[:,1], 
               np.array(m2_max_model_emukit.X)[:,0], m2_max_model_emukit.Y, marker='o', color='red')

    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    ax.set_zlabel('Target function')
    plt.tight_layout()



In [ ]:
if run_grid_simulation:

    ## Heatmaps
    extents = [m2_space.parameters[1].min, m2_space.parameters[1].max, 
               m2_space.parameters[0].min, m2_space.parameters[0].max]

    # True values
    fig, ax = plt.subplots()
    im = ax.imshow(m2_Z, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()

    # Model
    fig, ax = plt.subplots()
    im = ax.imshow(m2_mu_plot_pred2, extent=extents, aspect='auto', origin='lower')
    ax.set_title('Model (mean)')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    fig.colorbar(im, ax=ax)
    fig.show()


    # Difference
    fig, ax = plt.subplots()
    vmin = (m2_mu_plot_pred2-m2_Z).min()
    vmax = (m2_mu_plot_pred2-m2_Z).max()
    vmin_max = max(abs(vmin), abs(vmax))
    divnorm=colors.TwoSlopeNorm(vcenter=0., vmin=-vmin_max, vmax=vmin_max) # vmin=-5., vcenter=0., vmax=10
    im = ax.imshow(m2_mu_plot_pred2-m2_Z, extent=extents, aspect='auto', cmap="bwr", norm=divnorm, origin='lower')
    ax.set_title('Difference between model and target function')
    ax.set_xlabel(m2_param_2)
    ax.set_ylabel(m2_param_1)
    # Add points where simulation evaluated
#     ax.plot(m2_x[:,0], m2_x[:,1], 'ro')
#     ax.plot(m2_x[:,1], m2_x[:,0], 'ro')
    ax.plot(np.array(m2_max_model_emukit.X)[:,1], np.array(m2_max_model_emukit.X)[:,0], 'ro')
    fig.colorbar(im, ax=ax)
    fig.show()



In [ ]:
## Greedy maximization using the simulator
# # opt 1
from collections import namedtuple
Min_val = namedtuple('Min_val', 'fun x')
min_idx = np.argmin(m2_y_plot)
true_minim = Min_val( m2_y_plot[min_idx], m2_x_plot[min_idx])

print("True min value: ", m2_y_plot[min_idx])
print("True min location: ", m2_x_plot[min_idx])

# # # opt2
# nr_custom_params = 2
# wirte_output_txt = False
# from scipy.optimize import minimize


# # func_to_minimize = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2  
# def func_to_minimize(x):
#     print(x)
#     return neg_run_missile_sim(np.array(x).reshape(1,nr_custom_params))

# bnds = [(m2_domain_param_1),
#         (m2_domain_param_2),
# #         (m3_domain_param_3),
# #         (m3_domain_param_4),
# #         (m3_domain_param_5),
# #         (m3_domain_param_6)
#        ]


# initial_guess = [np.mean(m2_domain_param_1),
#                  np.mean(m2_domain_param_2),
# #                  np.mean(m3_domain_param_3),
# #                  np.mean(m3_domain_param_4),
# #                  np.mean(m3_domain_param_5),
# #                  np.mean(m3_domain_param_6)
#                 ]
# true_minim = minimize(func_to_minimize, initial_guess, bounds=bnds) # , method='SLSQP'constraints=cons



In [ ]:
min_val_from_sim = true_minim.fun
min_loc_from_sim = true_minim.x
min_val_from_emu = results.minimum_value
min_loc_from_emu = results.minimum_location

min_val_diff = min_val_from_sim - min_val_from_emu
min_loc_diff = min_loc_from_sim - min_loc_from_emu

print("Min val from sim - min val from em: \n", min_val_diff)
print('\n')
print("Min location from sim - min location from em: \n", min_loc_diff)
print('\n')
print('\n')
print("Min location from sim: \n", min_loc_from_sim)
print("Min location from emu: \n", min_loc_from_emu)
print('\n')
print('\n')
print("Min value from sim: \n", min_val_from_sim)
print("Min value from emu: \n", min_val_from_emu)